In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
args = pickle.load(open('arguments.pickle', 'rb'))

# Load data

In [3]:
n=0
for name in glob.glob(os.path.join(args.genome_dir, '*patient*.tsv')):
    df_temp = pd.read_csv(name,sep='\t')
    if n == 0:
        df = df_temp
    else:
        df = pd.concat([df,df_temp], axis=0)
    n+=1

In [4]:
data_df = df[df['Host']=='Human']
data_df.shape

(3406, 17)

In [5]:
data = data_df[['Accession ID', 'Clade', 'Lineage', 'Collection date']]
data

,Accession ID,Clade,Lineage,Collection date
0,EPI_ISL_1081922,GRY,B.1.1.7,2021-02-16
1,EPI_ISL_1081924,GRY,B.1.1.7,2021-02-17
2,EPI_ISL_1081926,G,B.1,2021-02-16
3,EPI_ISL_1081928,G,B.1.524,2021-02-16
4,EPI_ISL_1081930,GR,B.1.1.274,2021-02-16
...,...,...,...,...
3401,EPI_ISL_648816,O,B.6.6,2020-05-30
3402,EPI_ISL_648817,O,B.6.6,2020-05-31
3403,EPI_ISL_648818,O,B.6.6,2020-06-01
3404,EPI_ISL_648819,O,B.6.6,2020-06-01


In [6]:
data.loc[:, 'Collection date'] = pd.to_datetime(data['Collection date'])

data.loc[:, 'Year'] = data['Collection date'].dt.strftime('%Y')
data.loc[:, 'Month'] = data['Collection date'].dt.strftime('%m')
data.loc[:, 'Year-Month'] = data['Collection date'].dt.strftime('%Y-%m')

In [7]:
data['Accession ID'] = [i.split('_')[2] for i in data['Accession ID']]
data.sort_values(['Accession ID'])

,Accession ID,Clade,Lineage,Collection date,Year,Month,Year-Month
2000,1034260,GRY,B.1.1.7,2021-02-11,2021,02,2021-02
2001,1034261,GRY,B.1.1.7,2021-02-12,2021,02,2021-02
2002,1034262,GRY,B.1.1.7,2021-02-13,2021,02,2021-02
2003,1034263,GH,B.1.36.16,2021-02-11,2021,02,2021-02
2004,1034264,G,B.1.524,2021-02-11,2021,02,2021-02
...,...,...,...,...,...,...,...
2995,995302,GH,B.1.427,2021-02-09,2021,02,2021-02
2996,995303,G,B.1.524,2021-02-10,2021,02,2021-02
2997,995305,G,B.1.524,2021-02-10,2021,02,2021-02
2998,995306,GR,B.1.1,2021-02-09,2021,02,2021-02


In [12]:
who_name_df = pd.read_csv(args.predefined_label, index_col=0, sep='\t')

In [14]:
who_name_dict = who_name_df.T.to_dict('list')

In [17]:
# ใส่่ who name
who_name_list = []
for k in data['Lineage']:
    if k in list(who_name_dict.keys()):
        who_name_list += who_name_dict[k]
    else:
        who_name_list += ['Others']

In [19]:
data['WHO name'] = who_name_list

In [21]:
data.to_csv(os.path.join(args.preprocess_dir, 'clade_df.csv'), index=False)

In [14]:
set(data['Year-Month'])

{'2020-01',
 '2020-02',
 '2020-03',
 '2020-04',
 '2020-05',
 '2020-06',
 '2020-07',
 '2020-08',
 '2020-09',
 '2020-10',
 '2020-11',
 '2020-12',
 '2021-01',
 '2021-02',
 '2021-03',
 '2021-04',
 '2021-05',
 '2021-06'}

In [15]:
id_month = {}
for m in list(set(data['Year-Month'])):
    id_month[m] = data[data['Year-Month']==m]['Accession ID'].values

In [16]:
pickle.dump(id_month, open(os.path.join(args.preprocess_dir, 'id_month.pickle'), 'wb'))
id_month_key = sorted(list(id_month.keys()))

# change protein each month

In [17]:
summary_change_protein_df = pd.read_csv(os.path.join(args.preprocess_dir, 'summary_change_protein_df.csv')) 

In [18]:
gene_list = list(set(summary_change_protein_df['gene']))
gene_list

['NSP3',
 'NSP16',
 'M',
 'NSP8',
 'ORF6',
 'NSP12',
 'ORF3a',
 'NSP6',
 'ORF8',
 'NSP1',
 'ORF7a',
 'NSP10',
 'NSP14',
 'ORF7b',
 'NSP7',
 'NSP13',
 'S',
 'NSP4',
 'NSP2',
 'NSP15',
 'E',
 'NSP5',
 'N',
 'NSP9',
 'ORF10']

In [19]:
len(set(summary_change_protein_df['gene']))

25

In [20]:
protein_alarm = {}
for g in gene_list:
    change_protein_df = summary_change_protein_df.copy()
    change_protein_df = change_protein_df[change_protein_df['check']==False]
    change_protein_df = change_protein_df[change_protein_df['gene']==g]
    protein_m_df = {}
    i=0
    for m in id_month_key:
        protein_m_df[m] = change_protein_df[change_protein_df['id'].isin(id_month[m].astype(int))]
        protein_m_df[m] = protein_m_df[m][['id','position_protein','change_protein']]
        protein_m_df[m] = protein_m_df[m].groupby(['position_protein','change_protein']).count().sort_values(by='position_protein') 
        protein_m_df[m]['month'] = i+1
        protein_m_df[m]['gene'] = g
        protein_m_df[m]['total'] = len(id_month[m])

        if i==0:
            protein_alarm_df = protein_m_df[m]
        else:
            protein_alarm_df = pd.concat([protein_alarm_df,protein_m_df[m]])
        i+=1
    
    protein_alarm[g] = protein_alarm_df.reset_index()
    protein_alarm[g]['graph'] = [p + ' : '+ str(n) + ' ' for p,n in zip(protein_alarm[g]['change_protein'],protein_alarm[g]['id'])]

In [21]:
len(sorted(protein_alarm.keys()))

25

In [22]:
from lib import plot_data
fig = plot_data.alarm_plot(protein_alarm['S'],[614,484],'position_protein')

In [24]:
pickle.dump(protein_alarm, open(os.path.join(args.preprocess_dir, 'protein_alarm.pickle'), 'wb'))

# change neucleotide each month

In [25]:
snps_max_df = pd.read_csv(os.path.join(args.preprocess_dir, 'snps_max_df.csv'))
snps_max_df

,id,query,position,sbjct,start_end,length,change
0,1034260,C,241,T,18_29891,29874,C241T
1,1034260,C,27286,N,18_29891,29874,C27286N
2,1034260,T,27285,N,18_29891,29874,T27285N
3,1034260,A,27284,N,18_29891,29874,A27284N
4,1034260,A,27283,N,18_29891,29874,A27283N
...,...,...,...,...,...,...,...
113996,995307,A,1904,G,125_8278,8154,A1904G
113997,995307,C,241,T,125_8278,8154,C241T
113998,995307,C,17518,T,8874_19445,10572,C17518T
113999,995307,G,22577,S,19529_29703,10175,G22577S


In [26]:
neucleotide_df = snps_max_df[snps_max_df['sbjct']!='-']
neucleotide_df

,id,query,position,sbjct,start_end,length,change
0,1034260,C,241,T,18_29891,29874,C241T
1,1034260,C,27286,N,18_29891,29874,C27286N
2,1034260,T,27285,N,18_29891,29874,T27285N
3,1034260,A,27284,N,18_29891,29874,A27284N
4,1034260,A,27283,N,18_29891,29874,A27283N
...,...,...,...,...,...,...,...
113996,995307,A,1904,G,125_8278,8154,A1904G
113997,995307,C,241,T,125_8278,8154,C241T
113998,995307,C,17518,T,8874_19445,10572,C17518T
113999,995307,G,22577,S,19529_29703,10175,G22577S


In [27]:
neucleotide_m_df = {}
i=0
for m in id_month_key:
    print(m)
    neucleotide_m_df[m] = neucleotide_df[neucleotide_df['id'].isin(id_month[m].astype(int))]
    neucleotide_m_df[m] = neucleotide_m_df[m][['id','position','change']]
    neucleotide_m_df[m] = neucleotide_m_df[m].groupby(['position','change']).count().sort_values(by='position')   
    neucleotide_m_df[m]['month'] = i+1
    neucleotide_m_df[m]['total'] = len(id_month[m])
    if i==0:
        neucleotide_alarm_df = neucleotide_m_df[m]
    else:
        neucleotide_alarm_df = pd.concat([neucleotide_alarm_df,neucleotide_m_df[m]])
    i+=1
# 

2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06


In [28]:
neucleotide_alarm_df = neucleotide_alarm_df.reset_index()  
neucleotide_alarm_df['graph'] = [p + ' : '+ str(n) + ' ' for p,n in zip(neucleotide_alarm_df['change'],neucleotide_alarm_df['id'])]
neucleotide_alarm_df

,position,change,id,month,total,graph
0,190,C190T,1,1,13,C190T : 1
1,199,G199N,1,1,13,G199N : 1
2,261,G261K,1,1,13,G261K : 1
3,865,C865T,2,1,13,C865T : 2
4,2791,T2791C,1,1,13,T2791C : 1
...,...,...,...,...,...,...
17573,29865,A29865N,20,18,372,A29865N : 20
17574,29866,A29866N,18,18,372,A29866N : 18
17575,29867,T29867N,12,18,372,T29867N : 12
17576,29868,G29868N,9,18,372,G29868N : 9


In [29]:
neucleotide_alarm_df.to_csv(os.path.join(args.preprocess_dir, 'neucleotide_alarm_df.csv'), index=False) 

# deletion each month

In [30]:
deletion_df = snps_max_df[snps_max_df['sbjct']=='-']
deletion_df

,id,query,position,sbjct,start_end,length,change
32,1034260,A,28274,-,18_29891,29874,A28274-
44,1034260,T,11296,-,18_29891,29874,T11296-
45,1034260,T,11295,-,18_29891,29874,T11295-
46,1034260,T,11294,-,18_29891,29874,T11294-
47,1034260,T,11293,-,18_29891,29874,T11293-
...,...,...,...,...,...,...,...
113856,995301,T,11293,-,50_29709,29660,T11293-
113868,995301,T,11288,-,50_29709,29660,T11288-
113869,995301,C,11289,-,50_29709,29660,C11289-
113870,995301,T,11290,-,50_29709,29660,T11290-


In [31]:
del_m_df = {}
i=0
for m in id_month_key:
#     print(m)
    del_m_df[m] = deletion_df[deletion_df['id'].isin(id_month[m].astype(int))]
    del_m_df[m] = del_m_df[m][['id','position']]
    del_m_df[m] = del_m_df[m].groupby('position').count().sort_values(by='position')   
    del_m_df[m]['month'] = i+1
    del_m_df[m]['total'] = len(id_month[m])
    if i==0:
        del_alarm_df = del_m_df[m]
    else:
        del_alarm_df = pd.concat([del_alarm_df,del_m_df[m]])
    i+=1

In [32]:
del_alarm_df = del_alarm_df.reset_index()

In [33]:
del_alarm_df.to_csv(os.path.join(args.preprocess_dir, 'del_alarm_df.csv'), index=False) 

# insertion each month

In [34]:
insertion_df = pd.read_csv(os.path.join(args.preprocess_dir, 'insertion_df.csv'))

In [35]:
insertion_df

,id,position_ref,n_insert_base,start_end,start_new,end_new
0,1081930,28250.0,1.0,10_29801,10,29801
1,1081930,28251.0,1.0,10_29801,10,29801
2,1315627,28250.0,1.0,63_29676,63,29676
3,1315627,28251.0,1.0,63_29676,63,29676
4,1315628,28250.0,3.0,28123_29766,28123,29766
5,1519355,28262.0,4.0,89_29740,89,29740
6,1543939,28262.0,4.0,45_29792,45,29792
7,1620144,28262.0,4.0,50_29759,50,29759
8,1914659,28262.0,4.0,18_29787,18,29787
9,1914660,28262.0,4.0,22260_29793,22260,29793


In [36]:
insert_m_df = {}
i=0
for m in id_month_key:
#     print(m)
    insert_m_df[m] = insertion_df[insertion_df['id'].isin(id_month[m].astype(int))]
    insert_m_df[m] = insert_m_df[m][['id','position_ref','n_insert_base']]
    insert_m_df[m] = insert_m_df[m].groupby(['position_ref','n_insert_base']).count().sort_values(by='position_ref')   
    insert_m_df[m]['month'] = i+1
    insert_m_df[m]['total'] = len(id_month[m])
    if i==0:
        insert_alarm_df = insert_m_df[m]
    else:
        insert_alarm_df = pd.concat([insert_alarm_df,insert_m_df[m]])
    i+=1

In [37]:
insert_alarm_df = insert_alarm_df.reset_index()

In [38]:
insert_alarm_df

,position_ref,n_insert_base,id,month,total
0,28250.0,1.0,2,14,97
1,28251.0,1.0,2,14,97
2,28250.0,3.0,1,15,173
3,28262.0,4.0,6,16,401
4,28262.0,4.0,2,18,372


In [41]:
insert_alarm_df.to_csv(os.path.join(args.preprocess_dir, 'insert_alarm_df.csv'),index=False) 